In [30]:
import random
import pandas as pd

In [31]:
# Parameters for the Genetic Algorithm
POPULATION_SIZE = 50
NUM_GENERATIONS = 20
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.1

In [32]:
# Load the classrooms dataset
classrooms = pd.read_csv("/content/classrooms.csv")

In [33]:
classrooms.head()

,classroom_id,building_name,room_number,capacity,room_type
0,1,A,305,35,Lecture Hall
1,2,K,144,24,Classroom
2,3,B,710,46,Classroom
3,4,A,541,30,Classroom
4,5,I,747,35,Lecture Hall


In [34]:
# prompt: Using dataframe classrooms: generate 70 more row data

import pandas as pd
import numpy as np

# Number of rows to generate
num_rows = 70

# Create new data
new_data = {
    'classroom_id': np.arange(31, 31 + num_rows),  # Assuming classroom IDs are unique and sequential
    'building_name': np.random.choice(classrooms['building_name'].unique(), num_rows), #Randomly pick values from the existing column
    'room_number': np.random.randint(144, 959, num_rows), #Randomly generate numbers in the range of existing values
    'capacity': np.random.randint(21, 47, num_rows), #Randomly generate numbers in the range of existing values
    'room_type': np.random.choice(classrooms['room_type'].unique(), num_rows), #Randomly pick values from the existing column
}

# Create a new DataFrame
new_classrooms = pd.DataFrame(new_data)

# Concatenate the new DataFrame with the original DataFrame
classrooms = pd.concat([classrooms, new_classrooms], ignore_index = True)
# Display the updated DataFrame
classrooms.head(100)

,classroom_id,building_name,room_number,capacity,room_type
0,1,A,305,35,Lecture Hall
1,2,K,144,24,Classroom
2,3,B,710,46,Classroom
3,4,A,541,30,Classroom
4,5,I,747,35,Lecture Hall
...,...,...,...,...,...
95,96,B,459,24,Classroom
96,97,K,711,30,Classroom
97,98,F,825,28,Lecture Hall
98,99,D,860,26,Lecture Hall


In [35]:
# Sample course details (mocked for this context)
courses = [
    {"course_id": 1, "students": 30, "room_type": "Lecture Hall"},
    {"course_id": 2, "students": 20, "room_type": "Classroom"},
    {"course_id": 3, "students": 40, "room_type": "Classroom"},
    {"course_id": 4, "students": 35, "room_type": "Lecture Hall"},
]

In [36]:
# Step 1: Initialize Population
def initialize_population(courses, classrooms, population_size=POPULATION_SIZE):
    population = []
    for _ in range(population_size):
        chromosome = {
            course["course_id"]: random.choice(classrooms["classroom_id"].tolist())
            for course in courses
        }
        population.append(chromosome)
    return population

In [37]:
# Step 2: Fitness Function
def calculate_fitness(chromosome, courses, classrooms):
    fitness = 0
    classrooms_dict = classrooms.set_index("classroom_id").to_dict("index")

    for course in courses:
        course_id = course["course_id"]
        assigned_room_id = chromosome[course_id]
        room = classrooms_dict[assigned_room_id]

        # Check room type compatibility
        if course["room_type"] == room["room_type"]:
            fitness += 10

        # Check capacity compatibility
        if course["students"] <= room["capacity"]:
            fitness += 10

    return fitness

In [38]:
# Step 3: Selection
def select_parents(population, fitnesses):
    selected = random.choices(population, weights=fitnesses, k=2)
    return selected

In [39]:
# Step 4: Crossover
def crossover(parent1, parent2, crossover_rate=CROSSOVER_RATE):
    if random.random() > crossover_rate:
        return parent1.copy(), parent2.copy()
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = {**dict(list(parent1.items())[:crossover_point]), **dict(list(parent2.items())[crossover_point:])}
    child2 = {**dict(list(parent2.items())[:crossover_point]), **dict(list(parent1.items())[crossover_point:])}
    return child1, child2

In [40]:
# Step 5: Mutation
def mutate(chromosome, classrooms, mutation_rate=MUTATION_RATE):
    if random.random() < mutation_rate:
        course_to_mutate = random.choice(list(chromosome.keys()))
        chromosome[course_to_mutate] = random.choice(classrooms["classroom_id"].tolist())
    return chromosome

In [41]:
# Main Genetic Algorithm
def genetic_algorithm(courses, classrooms, generations=NUM_GENERATIONS):
    population = initialize_population(courses, classrooms)
    best_fitness_per_gen=[]
    best_solution_per_gen=[]
    for generation in range(generations):
        fitnesses = [calculate_fitness(chromosome, courses, classrooms) for chromosome in population]
        new_population = []
        for _ in range(len(population) // 2):
            parent1, parent2 = select_parents(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, classrooms))
            new_population.append(mutate(child2, classrooms))
        population = new_population
        # Best solution for current generation
        best_chromosome = max(population, key=lambda c: calculate_fitness(c, courses, classrooms))
        best_fitness = calculate_fitness(best_chromosome, courses, classrooms)
        best_fitness_per_gen.append(best_fitness)
        best_solution_per_gen.append(best_chromosome)
        # Display Generation output for all generations:
        print(f"Generation {generation + 1}") # generation + 1 is the actual generation number
        print(f"Best Fitness Score: {best_fitness}")
        print(f"Best Solution: {best_chromosome}")
        print("-" * 80)

    # Best solution overall:
    best_chromosome_overall = max(best_solution_per_gen, key=lambda c: calculate_fitness(c, courses, classrooms))
    return best_chromosome_overall

In [42]:
# Run the genetic algorithm with the dataset
best_schedule = genetic_algorithm(courses, classrooms)

Generation 1
Best Fitness Score: 70
Best Solution: {1: 3, 2: 45, 3: 66, 4: 42}
--------------------------------------------------------------------------------
Generation 2
Best Fitness Score: 80
Best Solution: {1: 36, 2: 45, 3: 66, 4: 42}
--------------------------------------------------------------------------------
Generation 3
Best Fitness Score: 80
Best Solution: {1: 36, 2: 40, 3: 66, 4: 42}
--------------------------------------------------------------------------------
Generation 4
Best Fitness Score: 80
Best Solution: {1: 5, 2: 76, 3: 45, 4: 60}
--------------------------------------------------------------------------------
Generation 5
Best Fitness Score: 80
Best Solution: {1: 5, 2: 94, 3: 66, 4: 42}
--------------------------------------------------------------------------------
Generation 6
Best Fitness Score: 80
Best Solution: {1: 56, 2: 94, 3: 45, 4: 42}
--------------------------------------------------------------------------------
Generation 7
Best Fitness Score: 80
B

In [43]:
# Display the best schedule
print("Best Schedule:", best_schedule)

Best Schedule: {1: 36, 2: 45, 3: 66, 4: 42}
